Loading Data

In [ ]:
# read data from .arff files
import os

import sktime
from sktime.datasets import load_from_tsfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# load data from .csv using pd
def load_data(path):

    # load data from .csv
    data = pd.read_csv(path, header=None)

    # get labels
    y = data.iloc[:, -1].values

    # get features
    X = data.iloc[:, :-1].values

    # return features and labels
    return X, y

Loading MITBIH DATA

In [ ]:
# Load mitbih data
train_x_mitbih, train_y_mitbih = load_data(
    "./datasets/ecg/mitbih_train.csv"
)
test_x_mitbih, test_y_mitbih = load_data(
    "./datasets/ecg/mitbih_test.csv"
)


MITBIH DATA VISUALIZATION

In [ ]:
# Train data mitbih

# create a dataframe from the data and labels 
train_df_mitbih = pd.DataFrame(train_x_mitbih)
train_df_mitbih['label'] = train_y_mitbih

# create a plot bar from the dataframe using pandas.plot.bar
train_df_mitbih['label'].value_counts().plot.bar(title = "Train data mitbih")

In [ ]:
# Test data mitbih

# create a dataframe from the data and labels 
test_df_mitbih = pd.DataFrame(test_x_mitbih)
test_df_mitbih['label'] = test_y_mitbih

# create a plot bar from the dataframe using pandas.plot.bar
test_df_mitbih['label'].value_counts().plot.bar(title = "Train data mitbih")

In [ ]:
# put test_df and train_df on the same plot bar with different colors 
train_df_mitbih['label'].value_counts().plot.bar(title = "Train and Test data", color = 'blue', alpha = 1)
test_df_mitbih['label'].value_counts().plot.bar(color = 'red', alpha = 1)


In [ ]:
# extract unique labels from the data
mitbih_labels = train_df_mitbih['label'].unique()
counts_train = train_df_mitbih['label'].value_counts()
counts_test= test_df_mitbih['label'].value_counts()


print(mitbih_labels)
print("----------------------------------")
print("TRAIN: "+  str(counts_train))
print("----------------------------------")
print("TEST: " + str(counts_test))

In [ ]:
# for each label, plot the data#
for label in mitbih_labels:
    current = train_df_mitbih[train_df_mitbih['label'] == label].iloc[0]

    # create a dataframe from the data and labels
    current_df = pd.DataFrame(current)
    current_df = current_df.drop('label', axis = 0)
    current_df.plot(title = str(label), figsize = (10, 5))
    

MEDIA SI DEVIATIA STANDARD PER UNITATE DE TIMP

In [ ]:
# Media si deviatia standard pt fiecare label pt mitbih
from matplotlib import pyplot as plt


for label in mitbih_labels:
    current = train_df_mitbih[train_df_mitbih['label'] == label]

    # create a dataframe from the data and labels
    current_df = pd.DataFrame(current)
    current_df = current_df.drop('label', axis = 1)
    current_df_mean = current_df.mean()
    current_df_std = current_df.std()
    current_df_mean.plot(title = str(label), figsize = (10, 5))
    current_df_std.plot(title = str(label), figsize = (10, 5))

    # end plot
    plt.legend(['mean', 'std'])
    plt.show()

    

3.2.2. Extragerea atributelor

In [ ]:
# Setam atributele default date de intrare MITBIH pentru train
X_train_std_mitbih = pd.DataFrame(train_x_mitbih)

X_train_std_mitbih


In [ ]:
# Setam atributele default date de intrare MITBIH pentru test
X_test_std_mitbih = pd.DataFrame(test_x_mitbih)

X_test_std_mitbih


In [ ]:
# Setam atributele statistice date de intrare MITBIH pentru datele de train
X_train_statistics_mitbih = pd.DataFrame()

# Mean features
X_train_statistics_mitbih['mean'] = X_train_std_mitbih.mean(axis=1)

# Standard deviation features
X_train_statistics_mitbih['std'] = X_train_std_mitbih.std(axis=1)

# Average absolute difference features
X_train_statistics_mitbih['avg_abs_diff'] = X_train_std_mitbih.mad(axis=1)

# Min features
X_train_statistics_mitbih['min'] = X_train_std_mitbih.min(axis=1)

# Max features
X_train_statistics_mitbih['max'] = X_train_std_mitbih.max(axis=1)

# Max-min features
X_train_statistics_mitbih['max-min'] = X_train_statistics_mitbih['max'] - X_train_statistics_mitbih['min']

# Median features
X_train_statistics_mitbih['median'] = X_train_std_mitbih.median(axis=1)

# Median absolute deviation features, don't use X_train_std_mitbih.mad(axis=1) because it's the average absolute difference
# X_train_statistics_mitbih['median_abs_dev'] = (X_train_std_mitbih - X_train_statistics_mitbih['median']).abs().median(axis=1)

# Interquartile range features
X_train_statistics_mitbih['interquartile_range'] = X_train_std_mitbih.quantile(0.75, axis=1) - X_train_std_mitbih.quantile(0.25, axis=1)

# Values above mean features
# X_train_statistics_mitbih['values_above_mean'] = (X_train_std_mitbih > X_train_statistics_mitbih['mean']).sum(axis=1)

# Number of peaks features
# X_train_statistics_mitbih['number_of_peaks'] = (X_train_std_mitbih.diff(axis=1) < 0).sum(axis=1)

# Skewnness features
X_train_statistics_mitbih['skewness'] = X_train_std_mitbih.skew(axis=1)

# Kurtosis features
X_train_statistics_mitbih['kurtosis'] = X_train_std_mitbih.kurtosis(axis=1)

# Energy features
X_train_statistics_mitbih['energy'] = (X_train_std_mitbih ** 2).sum(axis=1)

# Average of absolute values features
X_train_statistics_mitbih['avg_abs_val'] = X_train_std_mitbih.abs().mean(axis=1)

# Signal magnitude area features
X_train_statistics_mitbih['signal_magnitude_area'] = X_train_std_mitbih.abs().sum(axis=1)

# Compute FFT features
fft = np.abs(np.fft.fft(X_train_std_mitbih))

# make fft data frame
fft = pd.DataFrame(fft)

# FFT mean features
X_train_statistics_mitbih['fft_mean'] = fft.mean(axis=1)

# FFT standard deviation features
X_train_statistics_mitbih['fft_std'] = fft.std(axis=1)

# FFT average absolute difference features
X_train_statistics_mitbih['fft_avg_abs_diff'] = fft.mad(axis=1)

# FFT min features
X_train_statistics_mitbih['fft_min'] = fft.min(axis=1)

# FFT max features
X_train_statistics_mitbih['fft_max'] = fft.max(axis=1)

# FFT max-min features
X_train_statistics_mitbih['fft_max-min'] = X_train_statistics_mitbih['fft_max'] - X_train_statistics_mitbih['fft_min']

# FFT median features
X_train_statistics_mitbih['fft_median'] = fft.median(axis=1)

# FFT median absolute deviation features, don't use fft.mad(axis=1) because it's the average absolute difference
# X_train_statistics_mitbih['fft_median_abs_dev'] = (fft - X_train_statistics_mitbih['fft_median']).abs().median(axis=1)

# FFT interquartile range features
X_train_statistics_mitbih['fft_interquartile_range'] = fft.quantile(0.75, axis=1) - fft.quantile(0.25, axis=1)

# FFT values above mean features
# X_train_statistics_mitbih['fft_values_above_mean'] = (fft > X_train_statistics_mitbih['fft_mean']).sum(axis=1)

# FFT number of peaks features
# X_train_statistics_mitbih['fft_number_of_peaks'] = (fft.diff(axis=1) < 0).sum(axis=1)

# FFT skewnness features
X_train_statistics_mitbih['fft_skewness'] = fft.skew(axis=1)

# FFT kurtosis features
X_train_statistics_mitbih['fft_kurtosis'] = fft.kurtosis(axis=1)

# FFT energy features
X_train_statistics_mitbih['fft_energy'] = (fft ** 2).sum(axis=1)

# FFT average of absolute values features
X_train_statistics_mitbih['fft_avg_abs_val'] = fft.abs().mean(axis=1)

# FFT signal magnitude area features
X_train_statistics_mitbih['fft_signal_magnitude_area'] = fft.abs().sum(axis=1)

X_train_statistics_mitbih

In [ ]:
# Setam atributele statistice date de intrare MITBIH pentru datele de test
X_test_statistics_mitbih = pd.DataFrame()

# Mean features
X_test_statistics_mitbih['mean'] = X_test_std_mitbih.mean(axis=1)

# Standard deviation features
X_test_statistics_mitbih['std'] = X_test_std_mitbih.std(axis=1)

# Average absolute difference features
X_test_statistics_mitbih['avg_abs_diff'] = X_test_std_mitbih.mad(axis=1)

# Min features
X_test_statistics_mitbih['min'] = X_test_std_mitbih.min(axis=1)

# Max features
X_test_statistics_mitbih['max'] = X_test_std_mitbih.max(axis=1)

# Max-min features
X_test_statistics_mitbih['max-min'] = X_test_statistics_mitbih['max'] - X_test_statistics_mitbih['min']

# Median features
X_test_statistics_mitbih['median'] = X_test_std_mitbih.median(axis=1)

# Median absolute deviation features, don't use X_test_std_mitbih.mad(axis=1) because it's the average absolute difference
# X_test_statistics_mitbih['median_abs_dev'] = (X_test_std_mitbih - X_test_statistics_mitbih['median']).abs().median(axis=1)

# Interquartile range features
X_test_statistics_mitbih['interquartile_range'] = X_test_std_mitbih.quantile(0.75, axis=1) - X_test_std_mitbih.quantile(0.25, axis=1)

# Values above mean features
# X_test_statistics_mitbih['values_above_mean'] = (X_test_std_mitbih > X_test_statistics_mitbih['mean']).sum(axis=1)

# Number of peaks features
# X_test_statistics_mitbih['number_of_peaks'] = (X_test_std_mitbih.diff(axis=1) < 0).sum(axis=1)

# Skewnness features
X_test_statistics_mitbih['skewness'] = X_test_std_mitbih.skew(axis=1)

# Kurtosis features
X_test_statistics_mitbih['kurtosis'] = X_test_std_mitbih.kurtosis(axis=1)

# Energy features
X_test_statistics_mitbih['energy'] = (X_test_std_mitbih ** 2).sum(axis=1)

# Average of absolute values features
X_test_statistics_mitbih['avg_abs_val'] = X_test_std_mitbih.abs().mean(axis=1)

# Signal magnitude area features
X_test_statistics_mitbih['signal_magnitude_area'] = X_test_std_mitbih.abs().sum(axis=1)

# Compute FFT features
fft = np.abs(np.fft.fft(X_test_std_mitbih))

# make fft data frame
fft = pd.DataFrame(fft)

# FFT mean features
X_test_statistics_mitbih['fft_mean'] = fft.mean(axis=1)

# FFT standard deviation features
X_test_statistics_mitbih['fft_std'] = fft.std(axis=1)

# FFT average absolute difference features
X_test_statistics_mitbih['fft_avg_abs_diff'] = fft.mad(axis=1)

# FFT min features
X_test_statistics_mitbih['fft_min'] = fft.min(axis=1)

# FFT max features
X_test_statistics_mitbih['fft_max'] = fft.max(axis=1)

# FFT max-min features
X_test_statistics_mitbih['fft_max-min'] = X_test_statistics_mitbih['fft_max'] - X_test_statistics_mitbih['fft_min']

# FFT median features
X_test_statistics_mitbih['fft_median'] = fft.median(axis=1)

# FFT median absolute deviation features, don't use fft.mad(axis=1) because it's the average absolute difference
# X_test_statistics_mitbih['fft_median_abs_dev'] = (fft - X_test_statistics_mitbih['fft_median']).abs().median(axis=1)

# FFT interquartile range features
X_test_statistics_mitbih['fft_interquartile_range'] = fft.quantile(0.75, axis=1) - fft.quantile(0.25, axis=1)

# FFT values above mean features
# X_test_statistics_mitbih['fft_values_above_mean'] = (fft > X_test_statistics_mitbih['fft_mean']).sum(axis=1)

# FFT number of peaks features
# X_test_statistics_mitbih['fft_number_of_peaks'] = (fft.diff(axis=1) < 0).sum(axis=1)

# FFT skewnness features
X_test_statistics_mitbih['fft_skewness'] = fft.skew(axis=1)

# FFT kurtosis features
X_test_statistics_mitbih['fft_kurtosis'] = fft.kurtosis(axis=1)

# FFT energy features
X_test_statistics_mitbih['fft_energy'] = (fft ** 2).sum(axis=1)

# FFT average of absolute values features
X_test_statistics_mitbih['fft_avg_abs_val'] = fft.abs().mean(axis=1)

# FFT signal magnitude area features
X_test_statistics_mitbih['fft_signal_magnitude_area'] = fft.abs().sum(axis=1)

X_test_statistics_mitbih

Models

In [ ]:
# SVM with standard parameters MITBIH
svm_std_parameters_mitbih = SVC()
svm_std_parameters_mitbih.fit(X_train_std_mitbih, train_y_mitbih)

# print prediction results
predictions = svm_std_parameters_mitbih.predict(X_test_std_mitbih)
print(classification_report(test_y_mitbih, predictions))


           precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     18118
         1.0       0.96      0.56      0.71       556
         2.0       0.97      0.86      0.91      1448
         3.0       0.75      0.48      0.59       162
         4.0       1.00      0.91      0.95      1608

    accuracy                           0.97     21892
   macro avg       0.93      0.76      0.83     21892
weighted avg       0.97      0.97      0.97     21892

In [ ]:
# Variance Threshold
VARIANCE_THRESHOLD = 0
SELECT_PERCENTILE = 90


In [ ]:
sel = VarianceThreshold(threshold=VARIANCE_THRESHOLD)

# Variance Threshold MITBIH 
X_train_variance_mitbih = X_train_std_mitbih.copy()
X_test_variance_mitbih = X_test_std_mitbih.copy()

sel.fit_transform(X_train_variance_mitbih)
sel.fit_transform(X_test_variance_mitbih)

# transform X_train to ndarray
X_train_variance_mitbih = sel.transform(X_train_variance_mitbih)
X_test_variance_mitbih = sel.transform(X_test_variance_mitbih)

print(X_test_variance_mitbih.shape)

In [ ]:
# Select Percentile mitbih
X_train_percentile_mitbih = X_train_std_mitbih.copy()

X_test_percentile_mitbih = X_test_std_mitbih.copy()

# add columns name to X_test_percentile x1, x2, x3, , xn
X_test_percentile_mitbih.columns = [f'x{i}' for i in range(1, X_test_percentile_mitbih.shape[1] + 1)]

# add columns name to X_train_percentile x1, x2, x3, , xn
X_train_percentile_mitbih.columns = [f'x{i}' for i in range(1, X_train_percentile_mitbih.shape[1] + 1)]

X_train_percentile_mitbih = SelectPercentile(percentile=SELECT_PERCENTILE).fit(X_train_percentile_mitbih, train_y_mitbih)

train_features_names_mitbih = X_train_percentile_mitbih.get_feature_names_out()

X_train_percentile_mitbih = X_train_percentile_mitbih.transform(X_train_std_mitbih)

# Remove features using SelectPercentile from X_test_percentile using the same features
X_test_percentile_mitbih = X_test_percentile_mitbih[train_features_names_mitbih]

print(X_train_percentile_mitbih)


In [ ]:
# SVM with standard parameters MITBIH with variance threshold
svm_std_parameters_mitbih_variance = SVC()
svm_std_parameters_mitbih_variance.fit(X_train_variance_mitbih, train_y_mitbih)

# print prediction results
predictions = svm_std_parameters_mitbih_variance.predict(X_test_variance_mitbih)

print(classification_report(test_y_mitbih, predictions))

VARIANCE 0

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     18118
         1.0       0.96      0.56      0.71       556
         2.0       0.97      0.86      0.91      1448
         3.0       0.75      0.48      0.59       162
         4.0       1.00      0.91      0.95      1608

    accuracy                           0.97     21892
   macro avg       0.93      0.76      0.83     21892
weighted avg       0.97      0.97      0.97     21892

In [ ]:
# SVM with standard parameters MITBIH with percentile threshold
svm_std_parameters_mitbih_percentile = SVC()
svm_std_parameters_mitbih_percentile.fit(X_train_percentile_mitbih, train_y_mitbih)

# print prediction results
predictions = svm_std_parameters_mitbih_percentile.predict(X_test_percentile_mitbih)

print(classification_report(test_y_mitbih, predictions))

PERCENTILE 70

             precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     18118
         1.0       0.97      0.56      0.71       556
         2.0       0.96      0.88      0.92      1448
         3.0       0.80      0.53      0.64       162
         4.0       1.00      0.90      0.95      1608

    accuracy                           0.97     21892
   macro avg       0.94      0.77      0.84     21892
weighted avg       0.97      0.97      0.97     21892

PERCENTILE 30

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     18118
         1.0       0.95      0.55      0.70       556
         2.0       0.95      0.80      0.87      1448
         3.0       0.92      0.28      0.43       162
         4.0       1.00      0.90      0.94      1608

    accuracy                           0.96     21892
   macro avg       0.95      0.71      0.78     21892
weighted avg       0.96      0.96      0.96     21892

PERCENTILE 10

            precision    recall  f1-score   support

         0.0       0.94      0.99      0.97     18118
         1.0       0.84      0.51      0.63       556
         2.0       0.94      0.60      0.73      1448
         3.0       0.88      0.04      0.08       162
         4.0       0.97      0.86      0.91      1608

    accuracy                           0.94     21892
   macro avg       0.91      0.60      0.66     21892
weighted avg       0.94      0.94      0.93     21892

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'poly', 'sigmoid']} 

CV = 3

In [ ]:
# SVM with GridSearchCV MITBIH with all features

svm_model_all_features_mitbih = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

svm_model_all_features_mitbih.fit(X_train_std_mitbih, train_y_mitbih)

print(svm_model_all_features_mitbih.best_params_)

# print prediction results
predictions = svm_model_all_features_mitbih.predict(X_test_std_mitbih)

print(classification_report(test_y_mitbih, predictions))


In [ ]:
# SVM   with GridSearchCV MITBIH with variance threshold

svm_model_variance_mitbih = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv=CV)

svm_model_variance_mitbih.fit(X_train_variance_mitbih, train_y_mitbih)

print(svm_model_variance_mitbih.best_params_)

# print prediction results
predictions = svm_model_variance_mitbih.predict(X_test_variance_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
# SVM with GridSearchCV MITBIH with percentile threshold

svm_model_percentile_mitbih = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

svm_model_percentile_mitbih.fit(X_train_percentile_mitbih, train_y_mitbih)

print(svm_model_percentile_mitbih.best_params_)

# print prediction results

predictions = svm_model_percentile_mitbih.predict(X_test_percentile_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
# Random Forest with standard parameters MITBIH with all features

rf_std_parameters_mitbih = RandomForestClassifier()

rf_std_parameters_mitbih.fit(X_train_std_mitbih, train_y_mitbih)

# print prediction results
predictions = rf_std_parameters_mitbih.predict(X_test_std_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
# Random Forest with standard parameters MITBIH with variance threshold

rf_std_parameters_mitbih_variance = RandomForestClassifier()

rf_std_parameters_mitbih_variance.fit(X_train_variance_mitbih, train_y_mitbih)

# print prediction results

predictions = rf_std_parameters_mitbih_variance.predict(X_test_variance_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
# Random Forest with standard parameters MITBIH with percentile threshold

rf_std_parameters_mitbih_percentile = RandomForestClassifier()

rf_std_parameters_mitbih_percentile.fit(X_train_percentile_mitbih, train_y_mitbih)

# print prediction results

predictions = rf_std_parameters_mitbih_percentile.predict(X_test_percentile_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
param_grid = {'n_estimators': [100, 200, 300, 400],
                'max_features': ['sqrt', 'log2'],
                'max_depth' : [5,6,7],
                'criterion' :['gini', 'entropy']}

In [ ]:
# Random Forest with GridSearchCV MITBIH with all features

rf_model_all_features_mitbih = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3, cv=CV)

rf_model_all_features_mitbih.fit(X_train_std_mitbih, train_y_mitbih)

print(rf_model_all_features_mitbih.best_params_)

# print prediction results

predictions = rf_model_all_features_mitbih.predict(X_test_std_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
# Random Forest with GridSearchCV MITBIH with variance threshold

rf_model_variance_mitbih = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3, cv=CV)

rf_model_variance_mitbih.fit(X_train_variance_mitbih, train_y_mitbih)

print(rf_model_variance_mitbih.best_params_)

# print prediction results

predictions = rf_model_variance_mitbih.predict(X_test_variance_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
# Random Forest with GridSearchCV MITBIH with percentile threshold

rf_model_percentile_mitbih = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3, cv=2)

rf_model_percentile_mitbih.fit(X_train_percentile_mitbih, train_y_mitbih)

print(rf_model_percentile_mitbih.best_params_)

# print prediction results

predictions = rf_model_percentile_mitbih.predict(X_test_percentile_mitbih)

print(classification_report(test_y_mitbih, predictions))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
